## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "AES-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\AES\\AES-S'

In [21]:
atprogram(project_path, verbose=2)

Building file: .././gpio_init.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -mcmse -MD -MP -MF "gpio_init.d" -MT"gpio_init.d" -MT"gpio_init.o"   -o "gpio_init.o" ".././gpio_init.c" 
Finished building: .././gpio_init.c
Building file: .././atmel_start.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\

0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
if live_plot:
    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 5,
        "fig": fig,
        "window_title": "Experiment AES",
        "file_name_base": "experiment_aes"
    }
else:
    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_aes"
    }

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [22]:
data = []
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
#     data = dgilib.logger.log(config_dict["plot_xmax"],stop_fn)
    dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))


## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [23]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [24]:
encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [25]:
decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [26]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(encrypt_charge) - 1}")

MAX_AES_BLOCKS: 100


In [27]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [28]:
results = []
for y in [encrypt_charge, decrypt_charge, encrypt_time, decrypt_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 4.2860e-14
    reduced chi-square = 4.3735e-16
    Akaike info crit   = -3534.59979
    Bayesian info crit = -3529.38945
[[Variables]]
    slope:      3.2102e-08 +/- 4.5277e-12 (0.01%) (init = 0)
    intercept: -2.0787e-09 +/- 4.2140e-09 (202.73%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.868

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 3.7137e-14
    reduced chi-square = 3.7895e-16
    Akaike info crit   = -3548.93197
    Bayesian info crit = -3543.72163
[[Variables]]
    slope:      3.3322e-08 +/- 4.2148e-12 (0.01%) (init = 0)
    intercept:  5.7589e-09 +/- 3.9227e-09 (68.12%) (init = 1)
[[Correlations]] (unre

In [29]:
fig2 = plt.figure(figsize=(9, 6))
fig2.canvas.set_window_title("Analysis AES")

In [30]:
charge_color = 'r'
time_color = 'b'

In [31]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Total Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [32]:
lines = []
lines += ax1.plot(num_bytes, encrypt_charge, charge_color+'-', label='Encrypt Charge')
lines += ax1.plot(num_bytes, decrypt_charge, charge_color+'--', label='Decrypt Charge')
lines += ax2.plot(num_bytes, encrypt_time, time_color+'-', label='Encrypt Time')
lines += ax2.plot(num_bytes, decrypt_time, time_color+'--', label='Decrypt Time')
ax1.legend(handles=lines)
ax1.set_title(f"Encrypt Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} C per byte\n" +
             f"Decrypt Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} C per byte\n" +
             f"Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [33]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(encrypt_charge, open("aes_encrypt_charge.p", "wb"))
pickle.dump(decrypt_charge, open("aes_decrypt_charge.p", "wb"))

# Python script to do measurement

In [34]:
%%writefile experiment_aes.py

from os import getcwd, path
import pickle

from pydgilib_extra import *
from atprogram.atprogram import atprogram

def experiment_aes(project_root=getcwd(), verbose=1):
    project_path = path.join(project_root, "AES-S")
    atprogram(project_path, verbose=verbose)

    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_aes",
        "log_folder": project_root,
        "verbose": verbose-1
    }
    
    def stop_fn(logger_data):
        return all(logger_data.gpio.values[-1])
    
    if verbose:
        print(f"Start DGILibExtra with config: \n{config_dict}\n")

    data = []
    with DGILibExtra(**config_dict) as dgilib:
        dgilib.device_reset()
        dgilib.logger.log(1000,stop_fn)
        data = dgilib.data
    
    if verbose:
        print(f"DGILibExtra data: {data}")

    def stop_function(pin_values):
        return all(pin_values)

    encrypt_charge, encrypt_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)
    decrypt_charge, decrypt_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

    # Save Charge amount list into pickle file
    pickle.dump(encrypt_charge, open(path.join(project_root, "aes_encrypt_charge.p"), "wb"))
    pickle.dump(decrypt_charge, open(path.join(project_root, "aes_decrypt_charge.p"), "wb"))
    
    if verbose:
        print(f"Saved results in: {path.join(project_root)}")

if __name__ == "__main__":
   experiment_aes()

Overwriting experiment_aes.py
